$
\begin{aligned}
&i h_{t}+0.5 h_{x x}+|h|^{2} h=0, \quad x \in[-5,5], \quad t \in[0, \pi / 2] \\
&h(0, x)=2 \operatorname{sech}(x) \\
&h(t,-5)=h(t, 5) \\
&h_{x}(t,-5)=h_{x}(t, 5)
\end{aligned}
$

In [ ]:
import numpy as np
import torch
from torch import nn, optim, autograd
from torch.nn import functional as F
from pyDOE import lhs
import scipy.io
import matplotlib.pyplot as plt
import matplotlib as mpl
import matplotlib.gridspec as gridspec
%matplotlib inline
from mpl_toolkits.axes_grid1 import make_axes_locatable

from models_all import *

#Paper reproduction
torch.manual_seed(1234)
torch.cuda.manual_seed(1234)
np.random.seed(1234)

# Training data

In [ ]:
data = scipy.io.loadmat('./NLS.mat')

t = data['tt'].flatten()[:,None]
x = data['x'].flatten()[:,None]
Exact = data['uu']
Exact_u = np.real(Exact)
Exact_v = np.imag(Exact)
Exact_h = np.sqrt(Exact_u**2 + Exact_v**2)

X, T = np.meshgrid(x,t)

X_star = np.hstack((T.flatten()[:,None], X.flatten()[:,None]))
u_star = Exact_u.T.flatten()[:,None]
v_star = Exact_v.T.flatten()[:,None]
h_star = Exact_h.T.flatten()[:,None]

In [ ]:
X_star = torch.from_numpy(X_star).float()

In [ ]:
N_train = 20000 
N_bound = 100 

# idx_x = np.random.choice(x.shape[0], N_bound, replace=False)
# x0 = x[idx_x,:]
# u0 = Exact_u[idx_x,0:1]
# v0 = Exact_v[idx_x,0:1]

### Labeled Sample

In [ ]:
np.random.seed(1234)
index_t = (lhs(1,10))*len(t)
index_t = index_t.astype(int).reshape(10,)
t_star = t[index_t]
index_x = (lhs(1,10))*len(x)
index_x = index_x.astype(int).reshape(10,)
x_star = x[index_x]
t_x_star = np.hstack((t_star, x_star))

In [ ]:
u_star_ = Exact[index_x,index_t]
u_star_ = np.hstack((np.real(u_star_).reshape(-1,1), np.imag(u_star_).reshape(-1,1)))
n=10

In [ ]:
gan_data_x_t = t_x_star
gan_data_u = u_star_.reshape(-1,2)
gan_data_x_t = torch.from_numpy(gan_data_x_t).float()
gan_data_u = torch.from_numpy(gan_data_u).float()

In [ ]:
gan_data_x_t

In [ ]:
gan_data_u

### The Image With Labeled Sample

In [ ]:
gan_data_x_t = gan_data_x_t.cpu()
lb = np.array([-5.0, 0.0])
ub = np.array([5.0, np.pi/2])
fig, ax = plt.subplots()
gs0 = gridspec.GridSpec(1, 2)
gs0.update(top=1-0.06, bottom=1-1/3, left=0.15, right=0.85, wspace=0)
ax = plt.subplot(gs0[:, :])
ax.plot(gan_data_x_t[:,0:1], gan_data_x_t[:,1:2],  'kx',alpha=1,markersize = 4)
ax.set_xlabel('$t$')
ax.set_ylabel('$x$')
ax.set_title('$|h(t, x)|$', fontsize = 10)
h = ax.imshow(Exact_h, interpolation='nearest', cmap='rainbow', 
              extent=[lb[1], ub[1], lb[0], ub[0]], 
              origin='lower', aspect='auto')
divider = make_axes_locatable(ax)
cax = divider.append_axes("right", size="5%", pad=0.05)
fig.colorbar(h, cax=cax)
plt.show()

In [ ]:
Exact_h.shape

# Testing Data

In [ ]:
np.random.seed(5678)
index = lhs(1,2000)*len(X_star)
index = index.reshape(-1,).astype('int')

In [ ]:
test_x = X_star[index]
test_u = h_star[index]

In [ ]:
test_x

# Data For PINNs

In [ ]:
def sech(x):
    h = 2/(np.exp(x)+np.exp(-x))
    return h

In [ ]:
la = np.array([np.pi/2,5])
lb = np.array([0,-5])


t_limit = np.array([0,np.pi/2])
x_limit = np.array([-5,5])

np.random.seed(1234)
traindata_f = lb+(la-lb)*lhs(2,N_train)

traindata_f_t = traindata_f[:,0:1]
traindata_f_x = traindata_f[:,1:2]

np.random.seed(12345)
traindata_x_bound = x_limit[0]+(x_limit[1]-x_limit[0])*lhs(1,N_bound)
np.random.seed(123456)
traindata_t_bound = t_limit[0]+(t_limit[1]-t_limit[0])*lhs(1,N_bound)

data_h_0_x = np.concatenate((np.zeros(traindata_x_bound.shape),traindata_x_bound),axis = 1)

data_h_0_x_u = np.concatenate((2*sech(traindata_x_bound),np.zeros(traindata_x_bound.shape)),axis = 1)

data_h_t_5 = np.concatenate((traindata_t_bound,np.ones(traindata_t_bound.shape)*(5)),axis = 1)

data_h_t_fu_5 = np.concatenate((traindata_t_bound,np.ones(traindata_t_bound.shape)*(-5)),axis = 1)


data_5 = np.ones(traindata_t_bound.shape)*(5)

data_fu_5 = np.ones(traindata_t_bound.shape)*(-5)

traindata_f = torch.from_numpy(traindata_f).float()
traindata_f_t = torch.from_numpy(traindata_f_t).float()
traindata_f_x = torch.from_numpy(traindata_f_x).float()
data_h_0_x = torch.from_numpy(data_h_0_x).float()
traindata_t_bound = torch.from_numpy(traindata_t_bound).float()
data_5 = torch.from_numpy(data_5).float()
data_fu_5 = torch.from_numpy(data_fu_5).float()
data_h_t_5 = torch.from_numpy(data_h_t_5).float()
data_h_t_fu_5 = torch.from_numpy(data_h_t_fu_5).float()
data_h_0_x_u = torch.from_numpy(data_h_0_x_u).float()

# Method1
$\text { PINN }$

<div class="alert alert-info">`loss_function：`
$\mathrm{L}_{\text {PINN }}:=\mathrm{L}_f+\lambda_1 \mathrm{~L}_b \text { with } \mathrm{L}_b:=\sum_{i=1}^I \mathrm{~L}_{b_i}$</div>

In [ ]:
#t x  ; u v

In [ ]:
torch.manual_seed(1234)
torch.cuda.manual_seed(1234)
np.random.seed(1234)

PINNs1 = NN_H2(2, 100, 4, 2)
PINNs1.apply(weights_init)
optimizer1 = optim.Adam([{'params': PINNs1.parameters()}], lr=1e-3)
nIter2 = 20000

traindata_f_t.requires_grad_()
traindata_f_x.requires_grad_()
data_5.requires_grad_()
data_fu_5.requires_grad_()

#############gpu##############

traindata_t_bound = traindata_t_bound.cuda()
traindata_f = traindata_f.cuda()
traindata_f_t = traindata_f_t.cuda()
traindata_f_x = traindata_f_x.cuda()
data_h_0_x = data_h_0_x.cuda()
traindata_t_bound = traindata_t_bound.cuda()
data_5 = data_5.cuda()
data_fu_5 = data_fu_5.cuda()
data_h_t_5 = data_h_t_5.cuda()
data_h_t_fu_5 = data_h_t_fu_5.cuda()
data_h_0_x_u = data_h_0_x_u.cuda()
############gpu###############




loss_all_1 = []
test_1 = []

loss1_value = 1
it = 0
while loss1_value > 0.001:
    PINNs1 = PINNs1.cuda()
    ##### loss_Bi  ######
    h_0_x = PINNs1(data_h_0_x)
    h_t_5 = PINNs1(torch.cat((traindata_t_bound,data_5),1))
    h_t_fu_5 = PINNs1(torch.cat((traindata_t_bound,data_fu_5),1))
        
    loss_h_0_x = torch.mean(torch.square(h_0_x-data_h_0_x_u))
    loss_bound_h = torch.mean(torch.square(h_t_fu_5-h_t_5))
    grad_h_5 = autograd.grad(outputs=h_t_5, inputs=data_5,
                              grad_outputs=torch.ones_like(h_t_5),
                              create_graph=True, retain_graph=True, only_inputs=True)[0]
    grad_h_fu_5 = autograd.grad(outputs=h_t_fu_5, inputs=data_fu_5,
                              grad_outputs=torch.ones_like(h_t_fu_5),
                              create_graph=True, retain_graph=True, only_inputs=True)[0]
    
    loss_bound_h_x = torch.mean(torch.square(grad_h_5-grad_h_fu_5))
    
    ##### loss f  ######
    h = PINNs1(torch.cat((traindata_f_t, traindata_f_x), 1))
    u = h[:,0:1]
    v = h[:,1:2]
    u_x = autograd.grad(outputs=u, inputs=traindata_f_x,
                              grad_outputs=torch.ones_like(u),
                              create_graph=True, retain_graph=True, only_inputs=True)[0]
    u_xx = autograd.grad(outputs=u_x, inputs=traindata_f_x,
                              grad_outputs=torch.ones_like(u),
                              create_graph=True, retain_graph=True, only_inputs=True)[0]
    u_t = autograd.grad(outputs=u, inputs=traindata_f_t,
                              grad_outputs=torch.ones_like(u),
                              create_graph=True, retain_graph=True, only_inputs=True)[0]
    v_x = autograd.grad(outputs=v, inputs=traindata_f_x,
                              grad_outputs=torch.ones_like(v),
                              create_graph=True, retain_graph=True, only_inputs=True)[0]
    v_xx = autograd.grad(outputs=v_x, inputs=traindata_f_x,
                              grad_outputs=torch.ones_like(v),
                              create_graph=True, retain_graph=True, only_inputs=True)[0]    
    v_t = autograd.grad(outputs=v, inputs=traindata_f_t,
                              grad_outputs=torch.ones_like(v),
                              create_graph=True, retain_graph=True, only_inputs=True)[0]    
  
    f_u = -v_t+0.5*u_xx+(u**2+v**2)*u
    f_v = u_t+0.5*v_xx+(u**2+v**2)*v

    loss_f = torch.mean(torch.square(f_u))+torch.mean(torch.square(f_v))    
    
        
    #####loss PI#######
    loss = (loss_h_0_x+loss_bound_h+loss_bound_h_x)*2+loss_f    
    
    
    #######gpu#########
    loss = loss.cuda()
    
    #######gpu#########
    
    
    
    loss_all_1.append(loss.item())
    #loss = loss_f 
    loss1_value = loss.item()
    optimizer1.zero_grad()
    loss.backward()
    optimizer1.step()
    
    #########  test_loss NRMSE  #########
    PINNs1.cpu()
    e1 = relative_l2(np.sqrt(PINNs1(test_x).detach().numpy()[:,0:1]**2 + PINNs1(test_x).detach().numpy()[:,1:2]**2),test_u)
    test_1.append(e1)
    
    
    if it % 100 == 0:
        print('It:', it, 'Loss:', loss.item())
    it = it + 1        
loss1_value    

# Epochs

In [ ]:
Epochs = len(test_1)
NRMSE = relative_l2(np.sqrt(PINNs1(test_x).detach().numpy()[:,0:1]**2 + PINNs1(test_x).detach().numpy()[:,1:2]**2),test_u)

print('Epochs:',Epochs,'NRMSE:',NRMSE)

# Save data

In [ ]:
np.save('../experimental_data/method_1/test_loss_1',test_1)
np.save('../experimental_data/method_1/loss_all_1',loss_all_1)
torch.save(PINNs1,'../saved_model/PINNs1')